<big><b>Step 1:</b> Load the SQL extension</big>

In [37]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


<big><b>Step 2</b>: Connect sqlite to coursedata database</big>

In [2]:
%%sql

sqlite:///CourseData.db

'Connected: @CourseData.db'

<big><b>Step 3</b>: Determine columns within the dataset so that we can import this information to the appropriate tables </big>

In [42]:
%%sql

PRAGMA table_info(DATASET1);

 * sqlite:///CourseData.db
Done.


cid,name,type,notnull,dflt_value,pk
0,program_code,TEXT,0,None,0
1,program_name,TEXT,0,None,0
2,catalog_id,TEXT,0,None,0
3,course_title,TEXT,0,None,0
4,credits,TEXT,0,None,0
5,prereqs,TEXT,0,None,0
6,coreqs,TEXT,0,None,0
7,fees,TEXT,0,None,0
8,attributes,TEXT,0,None,0
9,description,TEXT,0,None,0


<big><b>Step 4:</b></big>

<big> A) Create INSTRUCTOR table </big>

In [38]:
%%sql 

DROP TABLE IF EXISTS INSTRUCTOR;
CREATE TABLE INSTRUCTOR(
    IID INTEGER PRIMARY KEY,
    Name TEXT NOT NULL
);

 * sqlite:///CourseData.db
Done.
Done.


[]

<big> B) Create LOCATION table </big>

In [39]:
%%sql

DROP TABLE IF EXISTS LOCATION;
CREATE TABLE LOCATION(
    LID INTEGER PRIMARY KEY,
    Location TEXT NOT NULL
);

 * sqlite:///CourseData.db
Done.
Done.


[]

<big> C) Create PROGRAM table</big>

In [40]:
%%sql

DROP TABLE IF EXISTS PROGRAM;
CREATE TABLE PROGRAM(
    PID INTEGER PRIMARY KEY,
    Program_Name TEXT NOT NULL
);
    

 * sqlite:///CourseData.db
Done.
Done.


[]

<big> D) Create CATALOG_COURSE table </big>

In [42]:
%%sql 

DROP TABLE IF EXISTS CATALOG_COURSE;
CREATE TABLE CATALOG_COURSE(
    CAID INTEGER NOT NULL PRIMARY KEY,
    CatalogID TEXT NOT NULL, 
    Program_Name TEXT NOT NULL,
    Credits VARCHAR NOT NULL,
    Title TEXT NOT NULL,
    PreReqs TEXT,
    Attributes TEXT,
    Description TEXT,
    FOREIGN KEY (Program_Name) REFERENCES PROGRAM(Program_Name)
); 


 * sqlite:///CourseData.db
Done.
Done.


[]

<big> E) Create OFFERINGS_COURSE table </big>

In [43]:
%%sql 

DROP TABLE IF EXISTS OFFERINGS_COURSE;
CREATE TABLE OFFERINGS_COURSE(
    COID INTEGER PRIMARY KEY,
    Name TEXT NOT NULL,
    CatalogID TEXT NOT NULL,
    CRN TEXT NOT NULL,
    Term TEXT NOT NULL,
    Section TEXT NOT NULL,
    Cap INTEGER NOT NULL,
    Act INTEGER NOT NULL,
    Rem INTEGER NOT NULL,
    FOREIGN KEY (Name) REFERENCES INSTRUCTOR (Name),
    FOREIGN KEY (CatalogID) REFERENCES CATALOG_COURSE (CatalogID)
);
    

 * sqlite:///CourseData.db
Done.
Done.


[]

<big> F) Create MEETINGS_COURSE table </big>

%%sql

DROP TABLE IF EXISTS MEETINGS_COURSE;
CREATE TABLE MEETINGS_COURSE(
    CMID INTEGER PRIMARY KEY,
    CRN TEXT NOT NULL,
    Location TEXT NOT NULL,
    Day TEXT NOT NULL,
    Start TEXT NOT NULL,
    End TEXT NOT NULL,
    FOREIGN KEY (Location) REFERENCES LOCATION(Location),
    FOREIGN KEY (CRN) REFERENCES OFFERINGS_COURSE(CRN)
);

<big> <b> Step 5: </b> Populating the database with data

Here we populated the data with that we have imported into the 3 dataset tables

-Dataset 1 <br>
-Dataset 2 <br>
-Dataset 3 <br>
   </big>

<big> A) Populate PROGRAM table </big>

In [47]:
%%sql

DELETE FROM PROGRAM;
INSERT INTO PROGRAM (Program_Name)
    SELECT DISTINCT program_name
    FROM DATASET1;
    

 * sqlite:///CourseData.db
0 rows affected.
83 rows affected.


[]

<big> B) Populate INSTRUCTOR table </big>

In [49]:
%%sql

DELETE FROM INSTRUCTOR;
INSERT INTO INSTRUCTOR (Name)
    SELECT DISTINCT primary_instructor
    FROM DATASET3;

 * sqlite:///CourseData.db
1105 rows affected.
1105 rows affected.


[]

<big> C) Populate LOCATION table </big>

In [50]:
%%sql

DELETE FROM LOCATION;
INSERT INTO LOCATION (Location)
    SELECT DISTINCT location
    FROM DATASET2;

 * sqlite:///CourseData.db
0 rows affected.
208 rows affected.


[]

<big> D) Populate CATALOG_COURSE table, link program name to PROGRAM table to ensure FK integrity </big>

In [51]:
%%sql

DELETE FROM CATALOG_COURSE;
INSERT INTO CATALOG_COURSE (CatalogID, Program_Name, Credits, Title, PreReqs, Attributes, Description)
    SELECT catalog_id, c.program_name, credits, course_title, prereqs, attributes, description
    FROM DATASET1 c
    LEFT JOIN PROGRAM d ON c.program_name = d.Program_Name;

 * sqlite:///CourseData.db
0 rows affected.
4441 rows affected.


[]

<big> D) Populate OFFERINGS_COURSE table, link instructor name to INSTRUCTOR and catalog_id to CATALOG_COURSE table to ensure FK integrity </big>

In [52]:
%%sql

DELETE FROM OFFERINGS_COURSE;
INSERT INTO OFFERINGS_COURSE(CRN, Term, CatalogID, Name, Section, Cap, Act, Rem)
    SELECT crn, term, c.catalog_id, section, IID, cap, act, rem
    FROM DATASET3 c
        INNER JOIN INSTRUCTOR i ON c.primary_instructor=i.Name
        INNER JOIN CATALOG_COURSE cc ON c.catalog_id = cc.CatalogID
 ;

 * sqlite:///CourseData.db
0 rows affected.
30504 rows affected.


[]

<big> E) Populate MEETINGS_COURSE table, link location to LOCATION and CRN to OFFERINGS_COURSE table to ensure FK integrity </big>

In [53]:
%%sql

DELETE FROM MEETINGS_COURSE;
INSERT INTO MEETINGS_COURSE (CRN, Location, Day, Start, End)
    SELECT c.crn, c.location, day, start, end
    FROM DATASET2 c
    LEFT JOIN LOCATION l ON c.location=l.Location
    LEFT JOIN OFFERINGS_COURSE oc ON c.crn = oc.CRN;


 * sqlite:///CourseData.db
0 rows affected.
758330 rows affected.


[]

In [54]:
%%sql

vacuum;

 * sqlite:///CourseData.db
Done.


[]